In [1]:
# will not copy to the main file
import os,sys,inspect
import pandas as pd
from memory_profiler import memory_usage

In [2]:
# will not copy to the main file
%load_ext rpy2.ipython
%load_ext memory_profiler

C:\Users\User\miniconda3\envs\525\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [3]:
import pyarrow.feather as feather

In [4]:
# will not copy to the main file
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir) # this refers to the project root folder

In [5]:
processed_folder = os.path.join(parentdir,"data", "processed")

input_path = os.path.join(processed_folder, "combined_data.csv")
output_path = os.path.join(processed_folder, "combined_data.feather")

In [6]:
%%time
%memit

df = pd.read_csv(input_path)

In [7]:
%%time
%memit

feather.write_feather(df, output_path)

In [8]:
# %%R

# install.packages("dplyr")
# library(arrow)

In [9]:
%%time
%%R
library(dplyr)
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("../data/processed/combined_data.feather")
print(class(r_table))
result <- r_table %>% count(model)
end_time <- Sys.time() 
print(result)
print(end_time - start_time)

R[write to console]: 
Attaching package: 'dplyr'


R[write to console]: The following objects are masked from 'package:stats':

    filter, lag


R[write to console]: The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: 'arrow'


R[write to console]: The following object is masked from 'package:utils':

    timestamp




[1] "tbl_df"     "tbl"        "data.frame"
# A tibble: 27 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# ... with 17 more rows
Time difference of 19.49337 secs
Wall time: 20.5 s


## Conclusion

After having thoroughly discussed, our team have reached a consensus that `Feather` file is the most suitable approach to transfer the dataframe from Python to R. We use read / write speed and the ability to support various operations as the criteria to make the decision.

As the input data is around 5.7GB, it is extremely slow to use `Pandas` exchange, hence its rejection. We do not select `Arrow` exchange as it only supports some operations (https://arrow.apache.org/docs/r/articles/dataset.html - this is given in his lecture note). `Parquet` file, even though is quite fast, is still slower than `Feather` V2 version when being read. Similarly, it is much faster to write into a `Feather` file than a `Parquet` file from a Python dataframe (https://ursalabs.org/blog/2020-feather-v2/ - this is given at the end of his lecture note as well). Even though a `Parquet` file can save more storage space, it is not a main concern for us as storage cost is cheap given this file size.